In [1]:
!pip install gql requests pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.9 MB/s eta 0:00:00


In [2]:
import pandas as pd

# Load the uploaded file (already in /mnt/data)
wallets_df = pd.read_excel('/content/Wallet id.xlsx')
wallets_df.columns = ['wallet_id']
wallets_df.dropna(inplace=True)
wallets_df.head()


,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [5]:
import requests
import time

SUBGRAPH_URL = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

def run_query(query):
    response = requests.post(SUBGRAPH_URL, json={'query': query})
    while response.status_code != 200:
        time.sleep(1)
        response = requests.post(SUBGRAPH_URL, json={'query': query})
    return response.json()

def get_events(wallet):
    wallet = wallet.lower()
    features = {
        'num_borrows': 0,
        'num_repays': 0,
        'num_liquidations': 0,
        'last_activity_ts': 0
    }

    borrow_query = f"""
    {{
      borrowEvents(where: {{borrower: "{wallet}"}}, first: 1000, orderBy: blockTimestamp, orderDirection: desc) {{
        amount
        blockTimestamp
      }}
    }}
    """
    repay_query = f"""
    {{
      repayEvents(where: {{payer: "{wallet}"}}, first: 1000, orderBy: blockTimestamp, orderDirection: desc) {{
        amount
        blockTimestamp
      }}
    }}
    """
    liquidation_query = f"""
    {{
      liquidationEvents(where: {{liquidator: "{wallet}"}}, first: 1000, orderBy: blockTimestamp, orderDirection: desc) {{
        blockTimestamp
      }}
    }}
    """

    borrow_data = run_query(borrow_query)
    repay_data = run_query(repay_query)
    liquidation_data = run_query(liquidation_query)

    borrows = borrow_data.get('data', {}).get('borrowEvents', [])
    repays = repay_data.get('data', {}).get('repayEvents', [])
    liquidations = liquidation_data.get('data', {}).get('liquidationEvents', [])


    features['num_borrows'] = len(borrows)
    features['num_repays'] = len(repays)
    features['num_liquidations'] = len(liquidations)

    all_timestamps = [int(b['blockTimestamp']) for b in borrows + repays + liquidations]
    if all_timestamps:
        features['last_activity_ts'] = max(all_timestamps)

    return features

In [6]:
from datetime import datetime
import time

wallet_features = []

print("Fetching data for wallets...")

for i, wallet in enumerate(wallets_df['wallet_id']):
    print(f"[{i+1}/{len(wallets_df)}] Processing wallet: {wallet}")
    features = get_events(wallet)
    features['wallet_id'] = wallet
    wallet_features.append(features)
    time.sleep(0.5)  # rate limit

features_df = pd.DataFrame(wallet_features)
features_df.head()


Fetching data for wallets...
[1/103] Processing wallet: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
[2/103] Processing wallet: 0x06b51c6882b27cb05e712185531c1f74996dd988
[3/103] Processing wallet: 0x0795732aacc448030ef374374eaae57d2965c16c
[4/103] Processing wallet: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
[5/103] Processing wallet: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
[6/103] Processing wallet: 0x104ae61d8d487ad689969a17807ddc338b445416
[7/103] Processing wallet: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
[8/103] Processing wallet: 0x124853fecb522c57d9bd5c21231058696ca6d596
[9/103] Processing wallet: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
[10/103] Processing wallet: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
[11/103] Processing wallet: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
[12/103] Processing wallet: 0x19df3e87f73c4aaf4809295561465b993e102668
[13/103] Processing wallet: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
[14/103] Processing wallet: 0x1c1b30ca93ef57452d53885d97a

,num_borrows,num_repays,num_liquidations,last_activity_ts,wallet_id
0,0,0,0,0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0,0,0,0,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0,0,0,0,0x0795732aacc448030ef374374eaae57d2965c16c
3,0,0,0,0,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0,0,0,0,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [10]:
# Convert timestamp to inactivity days
current_ts = int(datetime.now().timestamp())
features_df['inactivity_days'] = (current_ts - features_df['last_activity_ts']) / (60 * 60 * 24)

# Normalize features using Min-Max Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features_df[['num_borrows', 'num_repays', 'num_liquidations', 'inactivity_days']])

scaled_df = pd.DataFrame(scaled_features, columns=['borrows_scaled', 'repays_scaled', 'liquid_scaled', 'inactivity_scaled'])

# Compute final score (0–1000 scale)
# Lower liquidation & inactivity, higher borrows & repays -> higher score
features_df['score'] = (
    (0.3 * scaled_df['repays_scaled']) +
    (0.3 * scaled_df['borrows_scaled']) +
    (0.2 * (1 - scaled_df['liquid_scaled'])) +
    (0.2 * (1 - scaled_df['inactivity_scaled']))
) * 1000

# Final score rounded
features_df['score'] = features_df['score'].round().astype(int)


In [11]:
output_df = features_df[['wallet_id', 'score']]
output_df.to_csv('wallet_scores.csv', index=False)
output_df.head()


,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,400
1,0x06b51c6882b27cb05e712185531c1f74996dd988,400
2,0x0795732aacc448030ef374374eaae57d2965c16c,400
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,400
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,400


In [16]:
from google.colab import files

# Download the output CSV file
files.download('wallet_scores.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Wallet Risk Scoring using Compound V2 Protocol

## ✅ Objective:
To evaluate the risk profile of 100 wallet addresses based on their interaction history with the Compound V2 lending protocol.

## 📥 Data Collection:
Used [The Graph](https://thegraph.com/) to query Compound V2 Subgraph for:
- Borrow Events
- Repay Events
- Liquidation Events

## ⚙️ Features Extracted:
- Number of borrows (`num_borrows`)
- Number of repays (`num_repays`)
- Number of liquidations (`num_liquidations`)
- Inactivity (days since last transaction)

## 🧠 Scoring Logic:
Risk Score (0-1000) is calculated using:
- Repays and Borrows (positive influence)
- Liquidations and Inactivity (negative influence)
- Normalized via Min-Max Scaling
- Final Score = Weighted sum

## 📤 Output:
A CSV file with:
| wallet_id | score |
|-----------|-------|
| 0xfaa...f2 | 732   |